**Preprocessing**

In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler

plt.rcParams["figure.figsize"] = (10, 4)

df_raw = pd.read_csv(
    "AirQualityUCI.csv",
    sep=";",
    decimal=",",
    encoding="latin1"
)

print("Raw shape:", df_raw.shape)

df = df_raw.drop(columns=[c for c in df_raw.columns if "Unnamed" in c],
                 errors="ignore")

# HANDLING MISSING DATA
df = df.replace(-200, np.nan)

# Create timestamp from Date + Time
df["timestamp"] = pd.to_datetime(
    df["Date"] + " " + df["Time"],
    format="%d/%m/%Y %H.%M.%S",
    errors="coerce"
)

# Drop rows where timestamp can't be parsed
df = df.dropna(subset=["timestamp"])

# Sort and set as index
df = df.sort_values("timestamp").set_index("timestamp")

# Drop original Date/Time columns
df = df.drop(columns=["Date", "Time"])

print("After timestamp + sorting:", df.shape)

# DROP NMHC(GT)
df = df.drop(columns=["NMHC(GT)"], errors="ignore")

# IMPUTE REMAINING NaNs
numeric_cols = df.select_dtypes(include=[np.number]).columns

# Median imputation
medians = df[numeric_cols].median()
df[numeric_cols] = df[numeric_cols].fillna(medians)

# CREATE DERIVED TIME FEATURES
df["hour"] = df.index.hour
df["weekday"] = df.index.weekday   # 0 = Monday, 6 = Sunday
df["month"] = df.index.month
df["is_weekend"] = (df["weekday"] >= 5).astype(int)

# Cyclical encoding
df["hour_sin"] = np.sin(2 * np.pi * df["hour"] / 24)
df["hour_cos"] = np.cos(2 * np.pi * df["hour"] / 24)
df["weekday_sin"] = np.sin(2 * np.pi * df["weekday"] / 7)
df["weekday_cos"] = np.cos(2 * np.pi * df["weekday"] / 7)

# LAGS & MOVING AVERAGES
pollutants = ["CO(GT)", "C6H6(GT)", "NOx(GT)", "NO2(GT)"]
pollutants = [p for p in pollutants if p in df.columns]

# hours
lags = [1, 6, 12, 24]
# moving avgs
ma_windows = [6, 24]

for col in pollutants:
    for lag in lags:
        df[f"{col}_lag_{lag}h"] = df[col].shift(lag)

    for w in ma_windows:
        df[f"{col}_ma_{w}h"] = df[col].rolling(window=w, min_periods=1).mean()


# SCALING FEATURES
numeric_cols = df.select_dtypes(include=[np.number]).columns

scaler = StandardScaler()
df[numeric_cols] = scaler.fit_transform(df[numeric_cols])
df_model = df.dropna()

print("Final modelling dataset shape:", df_model.shape)
df_model.head()

df_model.to_csv("AirQuality_cleaned.csv", index=True)
print("AirQuality_cleaned.csv has been saved for future use.")

Raw shape: (9471, 17)
After timestamp + sorting: (9357, 13)
Final modelling dataset shape: (9333, 44)
AirQuality_cleaned.csv has been saved for future use.


**Main Code**

In [10]:
# ==========================================
# 1. INSTALL & IMPORTS
# ==========================================
# Install required packages silently
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
# Import all necessary metrics
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
from xgboost import XGBRegressor
import warnings

# Suppress warnings for cleaner output
warnings.filterwarnings('ignore')
# Set seaborn theme for cleaner plots and tables
sns.set_theme(style="whitegrid")
# Set pandas option to display numbers nicely in the final table
pd.set_option('display.float_format', '{:.3f}'.format)

# ==========================================
# 3. SPLITTING (TRAIN vs TEST)
# ==========================================
# NOTE: Assuming 'df' is already loaded in your environment.
train_df = df[df.index.year == 2004]
test_df = df[df.index.year == 2005]

# Define target columns
target_cols = ["CO(GT)", "C6H6(GT)", "NOx(GT)", "NO2(GT)"]
target_cols = [c for c in target_cols if c in df.columns]
feature_cols = [c for c in df.columns if c not in target_cols]

X_train = train_df[feature_cols]
y_train = train_df[target_cols]
X_test = test_df[feature_cols]
y_test = test_df[target_cols]

# Validation set for XGBoost (last 15% of training data)
val_split_idx = int(len(X_train) * 0.85)
X_train_sub = X_train.iloc[:val_split_idx]
y_train_sub = y_train.iloc[:val_split_idx]
X_val = X_train.iloc[val_split_idx:]
y_val = y_train.iloc[val_split_idx:]

print(f"📊 Split Info:")
print(f"   Train (2004): {len(X_train)} samples")
print(f"   Test  (2005): {len(X_test)} samples")

# ==========================================
# 4. MODEL DEFINITIONS
# ==========================================
models = {
    "LinearRegression": Pipeline([
        ('imputer', SimpleImputer(strategy='median')),
        ('regressor', LinearRegression())
    ]),
    "RandomForest": Pipeline([
        ('imputer', SimpleImputer(strategy='median')),
        ('regressor', RandomForestRegressor(n_estimators=100, n_jobs=-1, random_state=42))
    ]),
    "XGBoost": XGBRegressor(
        n_estimators=1000,
        learning_rate=0.05,
        max_depth=6,
        subsample=0.8,
        colsample_bytree=0.8,
        n_jobs=-1,
        random_state=42,
        tree_method="auto",
        missing=np.nan,
        early_stopping_rounds=50  # <--- FIXED: Moved here from .fit()
    )
}

# ==========================================
# 4.5 FOLDER CREATION LOGIC
# ==========================================
base_folder_name = "model_output"
output_folder = base_folder_name
counter = 1

# Check if folder exists, if so, increment number
while os.path.exists(output_folder):
    output_folder = f"{base_folder_name}_{counter}"
    counter += 1

os.makedirs(output_folder, exist_ok=True)
print(f"\n📂 Output Folder Created: {output_folder}")
print("   All plots and tables will be saved here silently.")

# ==========================================
# 5. TRAINING & EVALUATION LOOP
# ==========================================
results_list = []
# Dictionary to store predictions for final best model analysis
final_predictions = {}

print("\n🚀 Starting Training Loop...")

for name, model in models.items():
    print(f"Training {name}...")

    try:
        if name == "XGBoost":
            # FIXED: Removed early_stopping_rounds from here
            model.fit(
                X_train_sub, y_train_sub,
                eval_set=[(X_val, y_val)],
                verbose=False
            )
        else:
            model.fit(X_train, y_train)

        # Predict
        y_pred = model.predict(X_test)

        # Store predictions
        final_predictions[name] = y_pred

        # Ensure shape consistency
        if y_pred.ndim == 1:
            y_pred = y_pred.reshape(-1, 1)

        # Calculate Metrics & Plot per Target
        for i, target_col in enumerate(y_test.columns):
            y_true_col = y_test[target_col].values
            y_pred_col = y_pred[:, i]

            # Calculate Metrics (MAE, RMSE, R2)
            mae = mean_absolute_error(y_true_col, y_pred_col)
            rmse = np.sqrt(mean_squared_error(y_true_col, y_pred_col))
            r2 = r2_score(y_true_col, y_pred_col)

            results_list.append({
                "Model": name,
                "Pollutant": target_col,
                "MAE": mae,
                "RMSE": rmse,
                "R2": r2
            })

            # --- SILENT PLOTTING (Save and Close, No Show) ---
            clean_target = target_col.replace("(", "").replace(")", "")

            # 1. Residual Line Plot
            plt.figure(figsize=(10, 4))
            residuals = y_true_col - y_pred_col
            plt.plot(residuals, label='Residuals', alpha=0.6, linewidth=0.8, color='steelblue')
            plt.axhline(0, color='red', linestyle='--')
            plt.title(f"{name} - {target_col} - Residuals over Time")
            plt.ylabel("Error")
            plt.xlabel("Test Samples")
            plt.legend()
            plt.tight_layout()
            # Save and close immediately
            plt.savefig(os.path.join(output_folder, f"{name}_{clean_target}_Res_Line.png"))
            plt.close()

            # 2. Trend Plot (First 200 hours for visibility)
            plt.figure(figsize=(10, 4))
            plt.plot(y_true_col[:200], label='Actual', color='black', linewidth=1.5)
            plt.plot(y_pred_col[:200], label='Predicted', color='darkorange', alpha=0.8, linewidth=1.5)
            plt.title(f"{name} - {target_col} - Trend (First 200 Hrs)")
            plt.ylabel("Concentration")
            plt.legend()
            plt.tight_layout()
            # Save and close immediately
            plt.savefig(os.path.join(output_folder, f"{name}_{clean_target}_Trend.png"))
            plt.close()

    except Exception as e:
        print(f"⚠️ Failed to train {name}: {e}")

# ==========================================
# 6. FINAL SUMMARY TABLE (PRINTED)
# ==========================================
print("\n" + "="*50)
print("🏆 FINAL TEST SET PERFORMANCE TABLE")
print("="*50)
results_df = pd.DataFrame(results_list)

# Format table structure (MultiIndex on Pollutant, then Model)
summary_table = results_df.set_index(['Pollutant', 'Model']).sort_index()

# PRINT THE TABLE
print(summary_table)
print("="*50)

# Save table to CSV as well
table_path = os.path.join(output_folder, "final_performance_table.csv")
summary_table.to_csv(table_path)
print(f"\n✅ Table saved to CSV: {table_path}")


# ==========================================
# 7. BEST MODEL DETAILED PLOTS (SILENT SAVE)
# ==========================================
print("\nGenerating detailed plots for the optimal model (saving silently)...")

# Determine best model based on highest average R2 across all pollutants
avg_r2_per_model = results_df.groupby('Model')['R2'].mean()
best_model_name = avg_r2_per_model.idxmax()
best_model_score = avg_r2_per_model.max()

print(f"-> Optimal Algorithm selected: {best_model_name} (Avg R2: {best_model_score:.3f})")

# Retrieve predictions for the best model
best_preds_arr = final_predictions[best_model_name]

# Loop through targets to generate specific plots for the best model
for i, target_col in enumerate(y_test.columns):
    y_true_col = y_test[target_col].values
    y_pred_col = best_preds_arr[:, i]
    clean_target = target_col.replace("(", "").replace(")", "")

    # Calculate residuals
    residuals = y_true_col - y_pred_col

    # --- Plot A: Predicted vs True Scatter ---
    plt.figure(figsize=(6, 6))
    plt.scatter(y_true_col, y_pred_col, alpha=0.5, edgecolors='k', s=30, color='steelblue')
    # Add identity line
    fit_min = min(y_true_col.min(), y_pred_col.min())
    fit_max = max(y_true_col.max(), y_pred_col.max())
    plt.plot([fit_min, fit_max], [fit_min, fit_max], 'r--', lw=2, label='Perfect Prediction')
    plt.title(f"{best_model_name} - {target_col}\nPredicted vs True")
    plt.xlabel("True Value")
    plt.ylabel("Predicted Value")
    plt.legend()
    plt.tight_layout()
    # Save and close (silent)
    plt.savefig(os.path.join(output_folder, f"BEST_{best_model_name}_{clean_target}_PredVsTrue.png"))
    plt.close()

    # --- Plot B: Residual Distribution Histogram ---
    plt.figure(figsize=(8, 5))
    sns.histplot(residuals, kde=True, bins=30, color='steelblue', edgecolor='k')
    plt.axvline(0, color='red', linestyle='--', lw=2)
    plt.title(f"{best_model_name} - {target_col}\nResidual Distribution")
    plt.xlabel("Residual (True - Pred)")
    plt.ylabel("Count")
    plt.tight_layout()
    # Save and close (silent)
    plt.savefig(os.path.join(output_folder, f"BEST_{best_model_name}_{clean_target}_Res_Hist.png"))
    plt.close()

print(f"✅ All processes complete. All graphs saved in folder: {output_folder}")

📊 Split Info:
   Train (2004): 7110 samples
   Test  (2005): 2247 samples

📂 Output Folder Created: model_output_2
   All plots and tables will be saved here silently.

🚀 Starting Training Loop...
Training LinearRegression...
Training RandomForest...
Training XGBoost...

🏆 FINAL TEST SET PERFORMANCE TABLE
                             MAE  RMSE    R2
Pollutant Model                             
C6H6(GT)  LinearRegression 0.550 0.566 0.580
          RandomForest     0.063 0.087 0.990
          XGBoost          0.031 0.039 0.998
CO(GT)    LinearRegression 0.592 0.670 0.559
          RandomForest     0.284 0.424 0.823
          XGBoost          0.257 0.392 0.849
NO2(GT)   LinearRegression 0.361 0.496 0.815
          RandomForest     0.638 0.853 0.453
          XGBoost          0.493 0.723 0.607
NOx(GT)   LinearRegression 0.312 0.442 0.830
          RandomForest     0.332 0.476 0.803
          XGBoost          0.264 0.423 0.844

✅ Table saved to CSV: model_output_2/final_performance_table.c